Replicating Shapiro in Python. T-T

In [1]:
import pandas as pd
import numpy as np 
import statsmodels.api as sm
from scipy.stats import t as student_t
import tempfile
import os
from pathlib import Path

# User Inputs

In [2]:
foodenergy = 1  
smooth     = 0  
thresh1    = 0  
lag        = 12 
weight     = 0  
firstdiff  = 0  
level      = 1 
trend      = 0 
cumul      = 1  
quarterly  = 0  
hamilton   = 0

# *t-stat threshold
thresh2 =  0.15
thresh3 =  0.30

if level in [0, 1]: 
    if quarterly == 0:
        min_date = pd.to_datetime('1969-01')
    if quarterly == 1:
        min_date = pd.to_datetime('1969-01')

if level == 2: 
    if quarterly == 0:
        min_date = pd.to_datetime('1998-01')
    if quarterly == 1:
        min_date = pd.to_datetime('1998-01')

In [3]:
PCE_data = pd.read_stata(r"B:\OneDrive - DAZ\Desktop\Replicating Shapiro Final\raw_pce_data.dta")

# 790 - 830  

In [4]:
PCE_data['year'] = PCE_data['time_month'].dt.year
PCE_data['month'] = PCE_data['time_month'].dt.month
PCE_data['quarter'] = PCE_data['time_month'].dt.quarter

rename_map = {
    'cm_usna':        'pce_spend',
    'jcm_usna':       'pce',
    'jcxfem_usna':    'core',
    'cxfem_usna':     'core_spend',
    'jcgxfem_usna':   'core_goods',
    'jcsxem_usna':    'core_serv',
    'cgxfem_usna':    'core_goods_spend',
    'csxem_usna':     'core_serv_spend',
    'jnpcfm_usna':    'pcfm',
    'npcfm_usna':     'pcfm_spend',
    'npcfqm_usna':    'pcfm_quant',
    'cut_ip':     'util',
    'ip_ip':     'IP'
}

PCE_data = PCE_data.rename(columns=rename_map)

PCE_data.columns = [
     col.replace('jc', '').replace('_usna', '') if col.startswith('jc') and col.endswith('_usna') and not col.endswith('qm_usna')
    else col for col in PCE_data.columns
]

PCE_data.columns = [
    col[1:].replace('qm_usna', 'm_quant') if col.startswith('c') and col.endswith('qm_usna') else col
    for col in PCE_data.columns
]

PCE_data.columns = [
    col[1:].replace('_usna', '_spend') if col.startswith('c') and col.endswith('_usna') else col
    for col in PCE_data.columns
]

rename_map_2 = {
    'jnpcfm_usna':    'pcfm',
    'npcfm_usna':     'pcfm_spend',
    'npcfqm_usna':    'pcfm_quant'
}

PCE_data = PCE_data.rename(columns=rename_map_2)

for col in ['svxm_spend', 'novrm_spend']:
    if col in PCE_data.columns:
        PCE_data[col] = -PCE_data[col]

desired_order = [
    'time_month', 'pce', 'core', 'pce_spend',
    'core_goods', 'core_serv', 'core_goods_spend', 'core_serv_spend'
]

remaining = [col for col in PCE_data.columns if col not in desired_order]
PCE_data = PCE_data[desired_order + remaining]

PCE_data = PCE_data[PCE_data['core'].notna()]

dater = 'time_month' 
window = 120       

C:\Users\TonyGui\AppData\Local\Temp\ipykernel_13180\3078248364.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  PCE_data['year'] = PCE_data['time_month'].dt.year
C:\Users\TonyGui\AppData\Local\Temp\ipykernel_13180\3078248364.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  PCE_data['month'] = PCE_data['time_month'].dt.month
C:\Users\TonyGui\AppData\Local\Temp\ipykernel_13180\3078248364.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor 

In [5]:
PCE_data

,time_month,pce,core,pce_spend,core_goods,core_serv,core_goods_spend,core_serv_spend,util,IP,...,svtm,svxm,pcfm,pcfm_spend,pcfm_quant,ypdm_usna,ypm_usna,year,month,quarter
456,1959-01-01,15.164,15.501,306091.0,39.644,10.134,90513.0,131151.0,NaN,21.9616,...,15.022,11.176,48.390,4616.0,2.467,351.5,391.8,1959,1,1
457,1959-02-01,15.179,15.513,309554.0,39.650,10.145,93042.0,132440.0,NaN,22.3917,...,15.041,11.235,48.183,4659.0,2.498,352.9,393.7,1959,2,1
458,1959-03-01,15.189,15.531,312702.0,39.689,10.156,96513.0,133040.0,NaN,22.7142,...,15.060,11.235,48.524,4710.0,2.499,355.2,396.5,1959,3,1
459,1959-04-01,15.219,15.570,312193.0,39.769,10.183,95293.0,134042.0,NaN,23.1981,...,15.115,11.236,49.087,4751.0,2.474,358.2,399.9,1959,4,2
460,1959-05-01,15.227,15.589,316130.0,39.834,10.191,96432.0,135350.0,NaN,23.5476,...,15.152,11.236,49.170,4795.0,2.473,360.3,402.4,1959,5,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1241,2024-06-01,123.369,122.510,19747490.0,108.552,127.538,4298479.0,13213608.0,78.2170,103.2768,...,102.336,118.602,144.634,634639.0,110.685,21607.7,24626.7,2024,6,2
1242,2024-07-01,123.564,122.710,19859264.0,108.444,127.856,4335381.0,13277465.0,77.5825,102.5381,...,100.269,119.045,145.006,624192.0,108.575,21652.4,24695.1,2024,7,3
1243,2024-08-01,123.708,122.904,19898041.0,108.255,128.197,4311876.0,13352011.0,77.8849,103.0449,...,99.473,119.632,145.424,620481.0,107.614,21678.2,24740.2,2024,8,3
1244,2024-09-01,123.931,123.225,20027196.0,108.372,128.595,4372800.0,13418556.0,77.4246,102.5497,...,100.772,119.669,145.930,623651.0,107.786,21743.9,24814.6,2024,9,3


# 830 - 965

In [ ]:
if level in [0, 1]:
    PCE_data = PCE_data[PCE_data['year'] > 1958]

if level == 2:
    PCE_data = PCE_data[PCE_data['year'] > 1987]

"""Not sure about this range of column from the stata code"""
if quarterly == 1: 
    PCE_data = PCE_data.copy()

if level == 0:
    start = 'dfdm'

if level in [1, 2]:
    start = 'dbbm'


variables = ['util', 'IP', 'core', 'pce', 'core_goods', 'core_serv']
end = 'pcfm'
range_vars = PCE_data.loc[:, start:end].columns.tolist()
all_vars = variables + range_vars

if firstdiff == 0: 
    for col in all_vars:
        if col in PCE_data.columns:
            PCE_data[col + '_m'] = PCE_data[col].pct_change()
            PCE_data[col + '_plev'] = np.log(PCE_data[col])
            PCE_data[col] = PCE_data[col + '_m']
            PCE_data.drop(columns=[col + '_m'], inplace=True)

    for var in range_vars:
        quant_col = f"{var}_quant"
        if quant_col in PCE_data.columns:
            PCE_data[quant_col] = np.log(PCE_data[quant_col])


if firstdiff == 1: 
    for col in all_vars:
        if col in PCE_data.columns:
            PCE_data[quant_col] = np.log(PCE_data[quant_col])
            PCE_data[col + '_plev'] = np.log(PCE_data[col] / PCE_data[col].shift(1))
            PCE_data[col] = PCE_data[col + '_m']
            PCE_data.drop(columns=[col + '_m'], inplace=True)

    for var in range_vars:
        quant_col = f"{var}_quant"
        if quant_col in PCE_data.columns:
            PCE_data[f"{quant_col}_temp"] = PCE_data[quant_col]
            PCE_data[quant_col] = np.log(PCE_data[quant_col] / PCE_data[quant_col].shift(1))
            PCE_data.drop(columns=[f"{quant_col}_temp"], inplace=True)


if trend == 0:
    PCE_data['trend']  = 0
    PCE_data['trend2'] = 0
    PCE_data['trend3'] = 0


if trend == 1:
    time_var = 'time_month' if quarterly == 1 else 'time_quarter'
    PCE_data['trend']  = PCE_data[time_var]
    PCE_data['trend2'] = PCE_data[time_var] ** 2
    PCE_data['trend3'] = PCE_data[time_var] ** 3


def stata_tm(timestamp: pd.Timestamp) -> int:
    """
    Convert a pandas Timestamp to Stata's internal monthly date format.
    
    Parameters:
    - timestamp (pd.Timestamp): The input date
    
    Returns:
    - int: Number of months since Jan 1960 (Stata's tm format)
    """
    return 12 * (timestamp.year - 1960) + (timestamp.month - 1)

def stata_tq(timestamp: pd.Timestamp) -> int:
    """
    Convert a pandas Timestamp to Stata's internal quarterly date format.
    
    Parameters:
    - timestamp (pd.Timestamp): The input date
    
    Returns:
    - int: Number of quarters since 1960 Q1
    """
    year = timestamp.year
    quarter = (timestamp.month - 1) // 3 + 1
    return 4 * (year - 1960) + (quarter - 1)

if quarterly == 0: 
    PCE_data['time_month'] = PCE_data['time_month'].apply(stata_tm)
    PCE_data_date = PCE_data.copy()
    PCE_data_date = PCE_data_date[PCE_data_date['time_month'] >= stata_tm(min_date)]
    first = PCE_data_date['time_month'].min()
    last  = PCE_data_date['time_month'].max()

if quarterly == 1: 
    PCE_data['time_month'] = PCE_data['time_month'].apply(stata_tq)
    PCE_data_date = PCE_data.copy()
    PCE_data_date = PCE_data_date[PCE_data_date['time_month'] >= stata_tq(min_date)]
    first = PCE_data_date['time_month'].min()
    last  = PCE_data_date['time_month'].max()

lagc = cumul + lag - 1

end = 'pcfm'
var_range = PCE_data.loc[:, start:end].columns.tolist()

if hamilton == 1:
    for var in var_range:
        for suffix in ['quant', 'plev']:
            col = f"{var}_{suffix}"
            if col in PCE_data.columns:
                # Create lagged predictors from lag 24 to 36
                lagged_PCE_data_copy = pd.concat([PCE_data[col].shift(lag) for lag in range(24, 37)], axis=1)
                lagged_PCE_data_copy.columns = [f"{col}_lag{lag}" for lag in range(24, 37)]

                # Drop rows with missing lag values
                X = lagged_PCE_data_copy
                y = PCE_data[col]
                valid_idx = X.dropna().index.intersection(y.dropna().index)
                X = sm.add_constant(X.loc[valid_idx])
                y = y.loc[valid_idx]

                # Run regression and compute residuals
                model = sm.OLS(y, X).fit()
                residuals = y - model.fittedvalues

                # Replace original column with residuals
                PCE_data.loc[residuals.index, col] = residuals


In [ ]:
event_dum = first
t_stats, t2_stats = {}, {}  #i_t, #`i'_t2

demdumHH, demdumH, demdumL, demdum = {}, {}, {}, {}
supdumHH, supdumH, supdumL, supdum = {}, {}, {}, {}
ambdum = {}
demdumP, demdumN, supdumP, supdumN = {}, {}, {}, {}

while event_dum <= last:
    PCE_data_copy = PCE_data.copy()  
    PCE_data_copy['event'] = (PCE_data_copy[dater] == event_dum).astype(int)
    lower_bound = event_dum - window
    upper_bound = event_dum
    PCE_data_copy['ignore'] = (PCE_data_copy['time_month'] < lower_bound) | (PCE_data_copy['time_month'] > upper_bound)
    PCE_data_copy['temp'] = 0

    for var in var_range:
        quant_col = f"{var}_quant"
        plev_col = f"{var}_plev"
        if quant_col in PCE_data_copy.columns and plev_col in PCE_data_copy.columns:
            sub_df = PCE_data_copy[PCE_data_copy['ignore'] == 0].copy()
            lagged_quant = pd.concat([PCE_data_copy[quant_col].shift(lag) for lag in range(cumul, lagc + 1)], axis=1)
            lagged_quant.columns = [f"{quant_col}_lag{lag}" for lag in range(cumul, lagc + 1)]
            lagged_plev = pd.concat([PCE_data_copy[plev_col].shift(lag) for lag in range(cumul, lagc + 1)], axis=1)
            lagged_plev.columns = [f"{plev_col}_lag{lag}" for lag in range(cumul, lagc + 1)]
            trend_cols = [col for col in PCE_data_copy.columns if col.startswith("trend")]
            X = pd.concat([lagged_quant, lagged_plev, PCE_data_copy[trend_cols]], axis=1)
            X = sm.add_constant(X)
            y = PCE_data_copy[quant_col]
            valid_idx = X.dropna().index.intersection(y.dropna().index).intersection(sub_df.index)
            model = sm.OLS(y.loc[valid_idx], X.loc[valid_idx]).fit()
            PCE_data_copy.loc[valid_idx, f"{var}_uq"] = y.loc[valid_idx] - model.fittedvalues

        uq_col = f"{var}_uq"

        if smooth == 3 and uq_col in PCE_data_copy.columns:
                PCE_data_copy['test'] = PCE_data_copy[uq_col] + PCE_data_copy[uq_col].shift(1) + PCE_data_copy[uq_col].shift(2)
                PCE_data_copy[uq_col] = PCE_data_copy['test']
                PCE_data_copy.drop(columns=['test'], inplace=True)
                
        if smooth == 2 and uq_col in PCE_data_copy.columns:
            PCE_data_copy['test'] = PCE_data_copy[uq_col] + PCE_data_copy[uq_col].shift(1)
            PCE_data_copy[uq_col] = PCE_data_copy['test']
            PCE_data_copy.drop(columns=['test'], inplace=True)

        if smooth == 4 and uq_col in PCE_data_copy.columns:
            PCE_data_copy['test'] = (
                PCE_data_copy[uq_col] +
                PCE_data_copy[uq_col].shift(1) +
                PCE_data_copy[uq_col].shift(2) +
                PCE_data_copy[uq_col].shift(3)
            )
            PCE_data_copy[uq_col] = PCE_data_copy['test']
            PCE_data_copy.drop(columns=['test'], inplace=True)

        std = PCE_data_copy.loc[PCE_data_copy['ignore'] == 0, uq_col].std()
        PCE_data_copy['t'] = PCE_data_copy[uq_col] / std
        event_value = PCE_data_copy.loc[PCE_data_copy[dater] == event_dum, 't'].mean()
        t_stats[var] = event_value
        PCE_data_copy.drop(columns=['t'], inplace=True)

        uq_col = f"{var}_uq"
        up_col = f"{var}_up"
        quant_col = f"{var}_quant"
        plev_col = f"{var}_plev"

        sub_PCE_data_copy = PCE_data_copy[PCE_data_copy['ignore'] == 0].copy()
        lagged_quant = pd.concat([PCE_data_copy[quant_col].shift(lag) for lag in range(cumul, lagc + 1)], axis=1)
        lagged_quant.columns = [f"{quant_col}_lag{lag}" for lag in range(cumul, lagc + 1)]
        lagged_plev = pd.concat([PCE_data_copy[plev_col].shift(lag) for lag in range(cumul, lagc + 1)], axis=1)
        lagged_plev.columns = [f"{plev_col}_lag{lag}" for lag in range(cumul, lagc + 1)]
        trend_cols = [col for col in PCE_data_copy.columns if col.startswith("trend")]
        X = pd.concat([lagged_quant, lagged_plev, PCE_data_copy[trend_cols]], axis=1)
        X = sm.add_constant(X)
        y = PCE_data_copy[plev_col]
        valid_idx = X.dropna().index.intersection(y.dropna().index).intersection(sub_PCE_data_copy.index)
        model = sm.OLS(y.loc[valid_idx], X.loc[valid_idx]).fit()
        PCE_data_copy.loc[valid_idx, up_col] = y.loc[valid_idx] - model.fittedvalues

        if smooth == 3 and up_col in PCE_data_copy.columns:
            PCE_data_copy['test'] = PCE_data_copy[up_col] + PCE_data_copy[up_col].shift(1) + PCE_data_copy[up_col].shift(2)
            PCE_data_copy[up_col] = PCE_data_copy['test']
            PCE_data_copy.drop(columns=['test'], inplace=True)

        if smooth == 2 and up_col in PCE_data_copy.columns:
            PCE_data_copy['test'] = PCE_data_copy[up_col] + PCE_data_copy[up_col].shift(1)
            PCE_data_copy[up_col] = PCE_data_copy['test']
            PCE_data_copy.drop(columns=['test'], inplace=True)

        if smooth == 4 and up_col in PCE_data_copy.columns:
            PCE_data_copy['test'] = (
                PCE_data_copy[up_col] +
                PCE_data_copy[up_col].shift(1) +
                PCE_data_copy[up_col].shift(2) +
                PCE_data_copy[up_col].shift(3)
            )
            PCE_data_copy[up_col] = PCE_data_copy['test']
            PCE_data_copy.drop(columns=['test'], inplace=True)

        std = PCE_data_copy.loc[PCE_data_copy['ignore'] == 0, up_col].std()
        PCE_data_copy['t'] = PCE_data_copy[up_col] / std
        event_mean = PCE_data_copy.loc[PCE_data_copy[dater] == event_dum, 't'].mean()
        t2_stats[var] = event_mean 
        # PCE_data_copy.drop(columns=['t'], inplace=True)

        if weight == 1:
            uq_col = f"{var}_uq"
            up_col = f"{var}_up"
            lambda_col = f"lambda_{var}"
            PCE_data_copy[lambda_col] = PCE_data_copy[up_col] * PCE_data_copy[uq_col]
            valid = PCE_data_copy['ignore'] == 0
            lambda_std = PCE_data_copy.loc[valid, lambda_col].std()
            n_obs = PCE_data_copy.loc[valid, lambda_col].count()
            tlambda = PCE_data_copy[lambda_col] / lambda_std
            PCE_data_copy[f"{var}_tlambda"] = tlambda
            PCE_data_copy[f"{var}_weightD"] = student_t.cdf(tlambda, PCE_data_copy=n_obs - 2)
            PCE_data_copy[f"{var}_weightS"] = 1 - PCE_data_copy[f"{var}_weightD"]

        PCE_data_copy.drop(columns=[f"{var}_up", f"{var}_uq", "t"], inplace=True, errors='ignore')

    
    if foodenergy == 1: 
        for var in var_range:  
            t1 = t_stats.get(var)
            t2 = t2_stats.get(var)

            if t1 is None or t2 is None:
                continue  # skip if missing t or t2

            hh = (t1 < -thresh3 and t2 < -thresh3) or (t1 > thresh3 and t2 > thresh3)
            demdumHH[var] = hh

            h = ((t1 < -thresh2 and t2 < -thresh2) or (t1 > thresh2 and t2 > thresh2)) and not hh
            demdumH[var] = h

            l = ((t1 < -thresh1 and t2 < -thresh1) or (t1 > thresh1 and t2 > thresh1)) and not h and not hh
            demdumL[var] = l

            demdum[var] = hh or h or l
            PCE_data_copy[f"demdum_{var}"] = 0

            if demdum.get(var, False):
                PCE_data_copy.loc[PCE_data_copy[dater] == event_dum, f"demdum_{var}"] = 1

            shh = (t1 < -thresh3 and t2 > thresh3) or (t1 > thresh3 and t2 < -thresh3)
            sh  = ((t1 < -thresh2 and t2 > thresh2) or (t1 > thresh2 and t2 < -thresh2)) and not shh
            sl  = ((t1 < -thresh1 and t2 > thresh1) or (t1 > thresh1 and t2 < -thresh1)) and not sh and not shh

            supdumHH[var] = shh
            supdumH[var] = sh
            supdumL[var] = sl
            supdum[var] = shh or sh or sl

            PCE_data_copy[f"supdum_{var}"] = 0
            if supdum[var]:
                PCE_data_copy.loc[PCE_data_copy[dater] == event_dum, f"supdum_{var}"] = 1

            ambdum[var] = (
                not demdumHH[var] and not supdumHH[var] and
                not demdumH[var]  and not supdumH[var]  and
                not demdumL[var]  and not supdumL[var]
            )

            demdumP[var] = demdum[var] and (t2 > 0)
            demdumN[var] = demdum[var] and (t2 <= 0)
            supdumP[var] = supdum[var] and (t2 < 0)
            supdumN[var] = supdum[var] and (t2 >= 0)

            spend_col = f"{var}_spend"
            if spend_col in PCE_data_copy.columns and 'pce_spend' in PCE_data_copy.columns:
                # lagged contribution
                PCE_data_copy[f"contr_{var}m"] = (PCE_data_copy[spend_col].shift(1) / PCE_data_copy['pce_spend'].shift(1)) * PCE_data_copy[var]
                PCE_data_copy[f"weight_{var}m"] = PCE_data_copy[spend_col].shift(1) / PCE_data_copy['pce_spend'].shift(1)


    for name in [
    "demHH_contr_m", "demH_contr_m", "demL_contr_m",
    "supHH_contr_m", "supH_contr_m", "supL_contr_m",
    "amb_contr_m",
    "demP_contr_m", "demN_contr_m",
    "supP_contr_m", "supN_contr_m"]:
        PCE_data_copy[name] = 0


    for var in var_range:
        contrib_col = f"contr_{var}m"

        if contrib_col not in PCE_data_copy.columns:
            continue

        PCE_data_copy['demHH_contr_m'] += PCE_data_copy[contrib_col] * int(demdumHH.get(var, 0))
        PCE_data_copy['demH_contr_m']  += PCE_data_copy[contrib_col] * int(demdumH.get(var, 0))
        PCE_data_copy['demL_contr_m']  += PCE_data_copy[contrib_col] * int(demdumL.get(var, 0))

        PCE_data_copy['supHH_contr_m'] += PCE_data_copy[contrib_col] * int(supdumHH.get(var, 0))
        PCE_data_copy['supH_contr_m']  += PCE_data_copy[contrib_col] * int(supdumH.get(var, 0))
        PCE_data_copy['supL_contr_m']  += PCE_data_copy[contrib_col] * int(supdumL.get(var, 0))

        PCE_data_copy['amb_contr_m']   += PCE_data_copy[contrib_col] * int(ambdum.get(var, 0))

        PCE_data_copy['demP_contr_m']  += PCE_data_copy[contrib_col] * int(demdumP.get(var, 0))
        PCE_data_copy['demN_contr_m']  += PCE_data_copy[contrib_col] * int(demdumN.get(var, 0))

        PCE_data_copy['supP_contr_m']  += PCE_data_copy[contrib_col] * int(supdumP.get(var, 0))
        PCE_data_copy['supN_contr_m']  += PCE_data_copy[contrib_col] * int(supdumN.get(var, 0))


    if weight == 0: 
        PCE_data_copy["dem_contr_m"] = PCE_data_copy["demHH_contr_m"] + PCE_data_copy["demH_contr_m"] + PCE_data_copy["demL_contr_m"]
        PCE_data_copy["sup_contr_m"] = PCE_data_copy["supHH_contr_m"] + PCE_data_copy["supH_contr_m"] + PCE_data_copy["supL_contr_m"]


    if weight == 1: 
        PCE_data_copy["dem_contr_m"] = 0
        PCE_data_copy["sup_contr_m"] = 0

        for var in var_range:
            contr_col = f"contr_{var}m"
            weightD_col = f"{var}_weightD"
            weightS_col = f"{var}_weightS"

            if all(col in PCE_data_copy.columns for col in [contr_col, weightD_col, weightS_col]):
                PCE_data_copy["dem_contr_m"] += PCE_data_copy[contr_col] * PCE_data_copy[weightD_col]
                PCE_data_copy["sup_contr_m"] += PCE_data_copy[contr_col] * PCE_data_copy[weightS_col]


    for col in ["share_dempos", "share_demneg", "share_suppos", "share_supneg",
                "num_dempos", "num_demneg", "num_suppos", "num_supneg"]:
        PCE_data_copy[col] = 0


    if level == 2:
        if foodenergy == 1:
            k = 136
        elif foodenergy == 0:
            k = 124
    

    if level == 1:
        if foodenergy == 1:
            k = 129
        elif foodenergy == 0:
            k = 117


    for var in var_range:
            t2 = t2_stats.get(var)
            demdum_flag = demdum.get(var, False)
            supdum_flag = supdum.get(var, False)
            weight_col = f"weight_{var}m"

            if weight_col not in PCE_data_copy.columns:
                continue

            d = int(demdum_flag)
            s = int(supdum_flag)
            
            PCE_data_copy["share_dempos"] += d * (t2 > 0) * PCE_data_copy[weight_col]
            PCE_data_copy["share_demneg"] += d * (t2 < 0) * PCE_data_copy[weight_col]
            PCE_data_copy["share_suppos"] += s * (t2 < 0) * PCE_data_copy[weight_col]
            PCE_data_copy["share_supneg"] += s * (t2 > 0) * PCE_data_copy[weight_col]

            PCE_data_copy["num_dempos"] += d * (t2 > 0) * (1 / k)
            PCE_data_copy["num_demneg"] += d * (t2 < 0) * (1 / k)
            PCE_data_copy["num_suppos"] += s * (t2 < 0) * (1 / k)
            PCE_data_copy["num_supneg"] += s * (t2 > 0) * (1 / k)


    PCE_data_copy["dem_spend"] = 0
    PCE_data_copy["sup_spend"] = 0
    PCE_data_copy["dem_inf_m"] = 0
    PCE_data_copy["sup_inf_m"] = 0

    for var in var_range:
        spend_col = f"{var}_spend"
        if spend_col not in PCE_data_copy.columns:
            continue

        d = int(demdum.get(var, 0))
        s = int(supdum.get(var, 0))

        PCE_data_copy["dem_spend"] += d * PCE_data_copy[spend_col]
        PCE_data_copy["sup_spend"] += s * PCE_data_copy[spend_col]


    if foodenergy == 1:
        PCE_data_copy["tot_spend"] = PCE_data_copy["dem_spend"] + PCE_data_copy["sup_spend"]
        PCE_data_copy["dem_spend"] = PCE_data_copy.apply(
            lambda row: (row["dem_spend"] / row["tot_spend"]) * row["pce_spend"] if row["tot_spend"] != 0 else 0,
            axis=1)
        PCE_data_copy["sup_spend"] = PCE_data_copy.apply(
            lambda row: (row["sup_spend"] / row["tot_spend"]) * row["pce_spend"] if row["tot_spend"] != 0 else 0,
            axis=1)


    if foodenergy == 0: 
        PCE_data_copy["tot_spend"] = PCE_data_copy["dem_spend"] + PCE_data_copy["sup_spend"]

        PCE_data_copy["dem_spend"] = PCE_data_copy.apply(
            lambda row: (row["dem_spend"] / row["tot_spend"]) * row["core_spend"] if row["tot_spend"] != 0 else 0,
            axis=1)

        PCE_data_copy["sup_spend"] = PCE_data_copy.apply(
            lambda row: (row["sup_spend"] / row["tot_spend"]) * row["core_spend"] if row["tot_spend"] != 0 else 0,
            axis=1)


    for var in var_range:
        spend_col = f"{var}_spend"
        value_col = var
        if spend_col not in PCE_data_copy.columns or value_col not in PCE_data_copy.columns:
            continue

        PCE_data_copy[f"contrdem_{var}m"] = (PCE_data_copy[spend_col].shift(1) / PCE_data_copy["dem_spend"].shift(1)) * PCE_data_copy[value_col]
        PCE_data_copy[f"contrsup_{var}m"] = (PCE_data_copy[spend_col].shift(1) / PCE_data_copy["sup_spend"].shift(1)) * PCE_data_copy[value_col]


    for var in var_range:
        contrdem_col = f"contrdem_{var}m"
        contrsup_col = f"contrsup_{var}m"

        if contrdem_col in PCE_data_copy.columns and contrsup_col in PCE_data_copy.columns:
            d = int(demdum.get(var, 0))
            s = int(supdum.get(var, 0))

            PCE_data_copy["dem_inf_m"] += PCE_data_copy[contrdem_col] * d
            PCE_data_copy["sup_inf_m"] += PCE_data_copy[contrsup_col] * s


    cols_to_drop = [col for col in PCE_data_copy.columns if col.startswith("contrdem_") or col.startswith("contrsup_")]
    cols_to_drop.append("tot_spend")
    PCE_data_copy.drop(columns=cols_to_drop, inplace=True, errors="ignore")


    PCE_data_copy["demHH_contr_m"].attrs["label"] = "Very Likely Demand-driven PCE contribution"
    PCE_data_copy["demH_contr_m"].attrs["label"] = "Likely Demand-driven PCE contribution"
    PCE_data_copy["demL_contr_m"].attrs["label"] = "Possibly Demand-driven PCE contribution"
    PCE_data_copy["supHH_contr_m"].attrs["label"] = "Very Likely Supply-driven PCE contribution"
    PCE_data_copy["supH_contr_m"].attrs["label"] = "Likely Supply-driven PCE contribution"
    PCE_data_copy["supL_contr_m"].attrs["label"] = "Possibly Supply-driven PCE contribution"
    PCE_data_copy["amb_contr_m"].attrs["label"] = "Ambiguous PCE contribution"

    PCE_data_copy = PCE_data_copy[PCE_data_copy[dater] == event_dum].copy()
    script_dir = str(Path().resolve())
    file_path = os.path.join(script_dir) + "/"
    temp_folder = os.path.join(file_path, "loop_outputs")
    os.makedirs(temp_folder, exist_ok=True)
    filename = f"event_{event_dum}.csv"
    save_path = os.path.join(temp_folder, filename)
    PCE_data_copy.to_csv(save_path, index=False)

    event_dum += 1


# PCE_data_copy["demHH_contr_m"].attrs.get("label")

In [ ]:
script_dir = str(Path().resolve())
file_path = os.path.join(script_dir) + "/"
temp_folder = os.path.join(file_path, "loop_outputs")

outputs_combined = pd.read_csv(os.path.join(temp_folder, f"event_{first}.csv"))

for i in range(first + 1, last + 1):
    file_path = os.path.join(temp_folder, f"event_{i}.csv")
    if os.path.exists(file_path):
        df_next = pd.read_csv(file_path)
        outputs_combined = pd.concat([outputs_combined, df_next], ignore_index=True)


for col in [
    "demHH_contr_y", "demH_contr_y", "demL_contr_y",
    "supHH_contr_y", "supH_contr_y", "supL_contr_y",
    "amb_contr_y",
    "demP_contr_y", "demN_contr_y", "supP_contr_y", "supN_contr_y"
]:
    outputs_combined[col] = 1


if quarterly == 0: 
    end = 11
    end2 = 12


if quarterly == 1: 
    end = 3
    end2 = 4


outputs_combined = outputs_combined.sort_values(by="time_month").reset_index(drop=True)

for i in range(0, end + 1):
    for col in [
        "demHH_contr_y", "demH_contr_y", "demL_contr_y",
        "supHH_contr_y", "supH_contr_y", "supL_contr_y",
        "amb_contr_y", "demP_contr_y", "demN_contr_y", 
        "supP_contr_y", "supN_contr_y"]:
        monthly_col = col.replace("_y", "_m")
        if monthly_col in outputs_combined.columns:
            outputs_combined[col] *= (1 + outputs_combined[monthly_col].shift(i))


for col in [
    "demHH_contr_y", "demH_contr_y", "demL_contr_y",
    "supHH_contr_y", "supH_contr_y", "supL_contr_y",
    "amb_contr_y", "demP_contr_y", "demN_contr_y",
    "supP_contr_y", "supN_contr_y"]:

    outputs_combined[col] -= 1



outputs_combined["dem_contr_y"] = (
    outputs_combined["demHH_contr_y"] +
    outputs_combined["demH_contr_y"] +
    outputs_combined["demL_contr_y"]
)

outputs_combined["sup_contr_y"] = (
    outputs_combined["supHH_contr_y"] +
    outputs_combined["supH_contr_y"] +
    outputs_combined["supL_contr_y"]
)


for col in [
    "demHH_contr_y", "demH_contr_y", "demL_contr_y", "dem_contr_y",
    "supHH_contr_y", "supH_contr_y", "supL_contr_y", "sup_contr_y",
    "amb_contr_y", "demP_contr_y", "demN_contr_y", "supP_contr_y", "supN_contr_y"]:

    outputs_combined[col] *= 100




cols_to_copy = [
    "demHH_contr_m", "demH_contr_m", "demL_contr_m", "dem_contr_m",
    "supHH_contr_m", "supH_contr_m", "supL_contr_m", "sup_contr_m",
    "demP_contr_m", "demN_contr_m", "supP_contr_m", "supN_contr_m"]

for col in cols_to_copy:
    outputs_combined[f"{col}_na"] = outputs_combined[col]



cols_to_transform = [
    "demHH_contr_m", "demH_contr_m", "demL_contr_m", "dem_contr_m",
    "demP_contr_m", "demN_contr_m", "amb_contr_m",
    "supHH_contr_m", "supH_contr_m", "supL_contr_m", "sup_contr_m",
    "supP_contr_m", "supN_contr_m"]


outputs_combined["amb_contr_m_na"] = outputs_combined["amb_contr_m"]

for col in cols_to_transform:
    outputs_combined[col] = ((outputs_combined[col] + 1) ** end2 - 1) * 100


outputs_combined["dem_inf_y"] = 1
outputs_combined["sup_inf_y"] = 1


outputs_combined = outputs_combined.sort_values(by="time_month").reset_index(drop=True)

for i in range(0, end + 1):
    outputs_combined["dem_inf_y"] *= (1 + outputs_combined["dem_inf_m"].shift(i))
    outputs_combined["sup_inf_y"] *= (1 + outputs_combined["sup_inf_m"].shift(i))


outputs_combined["dem_inf_y"] = (outputs_combined["dem_inf_y"] - 1) * 100
outputs_combined["sup_inf_y"] = (outputs_combined["sup_inf_y"] - 1) * 100

outputs_combined["dem_inf_m_na"] = outputs_combined["dem_inf_m"] * 100
outputs_combined["sup_inf_m_na"] = outputs_combined["sup_inf_m"] * 100


outputs_combined["dem_inf_m"] = ((outputs_combined["dem_inf_m"] + 1) ** end2 - 1) * 100
outputs_combined["sup_inf_m"] = ((outputs_combined["sup_inf_m"] + 1) ** end2 - 1) * 100


if quarterly == 0: 
    outputs_combined["month_st"] = outputs_combined["month"].astype(str)
    outputs_combined["year_st"] = outputs_combined["year"].astype(str)

    outputs_combined["time"] = outputs_combined["year_st"] + "m" + outputs_combined["month_st"]
    outputs_combined["zero"] = 0



if quarterly == 1: 
    outputs_combined["quarter_st"] = outputs_combined["quarter"].astype(str)
    outputs_combined["year_st"] = outputs_combined["year"].astype(str)
    outputs_combined["time"] = outputs_combined["year_st"] + "q" + outputs_combined["quarter_st"]
    outputs_combined["zero"] = 0



In [9]:
script_dir = str(Path().resolve())
file_path = os.path.join(script_dir) + "/"
output_path = os.path.join(file_path, "final_output_from_shapiro_python.parquet")

outputs_combined.to_parquet(output_path, index=False)

In [ ]:
shaprio_python = pd.read_parquet(r"B:\OneDrive - DAZ\Desktop\Replicating Shapiro Final\final_output_from_shapiro_python.parquet")
shaprio_python

,time_month,pce,core,pce_spend,core_goods,core_serv,core_goods_spend,core_serv_spend,util,IP,...,supN_contr_m_na,amb_contr_m_na,dem_inf_y,sup_inf_y,dem_inf_m_na,sup_inf_m_na,month_st,year_st,time,zero
0,108,0.003912,0.003770,583479.0,0.002418,0.004709,180917.0,272374.0,0.002323,0.006049,...,0.002849,0.0,NaN,NaN,0.271792,0.451262,1,1969,1969m1,0
1,109,0.002544,0.003443,588675.0,0.003164,0.003553,182558.0,274872.0,0.002785,0.006400,...,0.001294,0.0,NaN,NaN,0.188305,0.290172,2,1969,1969m2,0
2,110,0.004697,0.004679,588862.0,0.004147,0.005047,181078.0,276007.0,0.004230,0.007840,...,0.002790,0.0,NaN,NaN,0.347701,0.570202,3,1969,1969m3,0
3,111,0.004406,0.003829,593870.0,0.003163,0.004273,183191.0,279048.0,-0.007235,-0.003679,...,0.002478,0.0,NaN,NaN,0.383123,0.473419,4,1969,1969m4,0
4,112,0.004012,0.004021,600271.0,0.001818,0.005449,183901.0,282744.0,-0.007325,-0.003778,...,0.003007,0.0,NaN,NaN,0.296997,0.453351,5,1969,1969m5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
665,773,0.001177,0.002217,19747490.0,0.001005,0.002610,4298479.0,13213608.0,0.001977,0.002885,...,0.001130,0.0,2.763610,2.241191,0.161921,0.078397,6,2024,2024m6,0
666,774,0.001581,0.001633,19859264.0,-0.000995,0.002493,4335381.0,13277465.0,-0.008112,-0.007153,...,0.001565,0.0,2.837743,2.243592,0.219982,0.134001,7,2024,2024m7,0
667,775,0.001165,0.001581,19898041.0,-0.001743,0.002667,4311876.0,13352011.0,0.003898,0.004943,...,0.001404,0.0,2.486529,2.153142,0.102052,0.124801,8,2024,2024m8,0
668,776,0.001803,0.002612,20027196.0,0.001081,0.003105,4372800.0,13418556.0,-0.005910,-0.004806,...,0.001196,0.0,2.553402,1.723338,0.391839,-0.053052,9,2024,2024m9,0


In [ ]:
shapiro_stata =  pd.read_stata(r"B:\OneDrive - DAZ\Desktop\Replicating Shapiro Final\Shapiro_STATA_output.dta")
shaprio_python = shaprio_python[shapiro_stata.columns]
shapiro_stata

,time_month,pce,core,pce_spend,core_goods,core_serv,core_goods_spend,core_serv_spend,util,IP,...,demN_contr_m_na,supP_contr_m_na,supN_contr_m_na,amb_contr_m_na,dem_inf_y,sup_inf_y,dem_inf_m_na,sup_inf_m_na,time,zero
0,1969-01-01,0.003912,0.003770,583479.0,0.002418,0.004709,180917.0,272374.0,0.002323,0.006049,...,0.000371,0.000196,0.002849,0.0,NaN,NaN,0.271792,0.451262,1969m1,0.0
1,1969-02-01,0.002544,0.003443,588675.0,0.003164,0.003553,182558.0,274872.0,0.002785,0.006400,...,-0.000103,0.000528,0.001292,0.0,NaN,NaN,0.188780,0.289968,1969m2,0.0
2,1969-03-01,0.004697,0.004679,588862.0,0.004147,0.005047,181078.0,276007.0,0.004230,0.007840,...,0.000833,0.000370,0.002790,0.0,NaN,NaN,0.347811,0.569921,1969m3,0.0
3,1969-04-01,0.004406,0.003829,593870.0,0.003163,0.004273,183191.0,279048.0,-0.007235,-0.003679,...,0.000533,0.000642,0.002478,0.0,NaN,NaN,0.382890,0.473471,1969m4,0.0
4,1969-05-01,0.004012,0.004021,600271.0,0.001818,0.005449,183901.0,282744.0,-0.007325,-0.003778,...,0.000387,0.000119,0.003007,0.0,NaN,NaN,0.296997,0.453351,1969m5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
665,2024-06-01,0.001177,0.002217,19747490.0,0.001005,0.002610,4298479.0,13213608.0,0.001977,0.002885,...,-0.000562,-0.000719,0.001130,0.0,2.763617,2.241206,0.161921,0.078397,2024m6,0.0
666,2024-07-01,0.001581,0.001633,19859264.0,-0.000995,0.002493,4335381.0,13277465.0,-0.008112,-0.007153,...,-0.000063,-0.000622,0.001565,0.0,2.837753,2.243578,0.219982,0.134001,2024m7,0.0
667,2024-08-01,0.001165,0.001581,19898041.0,-0.001743,0.002667,4311876.0,13352011.0,0.003898,0.004943,...,-0.000167,-0.000580,0.001404,0.0,2.486539,2.153146,0.102052,0.124801,2024m8,0.0
668,2024-09-01,0.001803,0.002612,20027196.0,0.001081,0.003105,4372800.0,13418556.0,-0.005910,-0.004806,...,0.000510,-0.001446,0.001196,0.0,2.553403,1.723337,0.391839,-0.053052,2024m9,0.0


In [94]:
haver_codes = pd.read_excel(r"B:\OneDrive - DAZ\Desktop\Replicating Shapiro Final\Haver Code to BEA Concordance.xlsx")
haver_codes

,PCE Category,Haver Code Expenditure,Haver Code Price,Haver Code Quantity,Shapiro Price Name,Shapiro Quantity Name,Shapiro Expenditure Name
0,Water transportation (65),cstiwm_usna,jcstiwm_usna,cstiwqm_usna,stiwm,stiwm_quant,stiwm_spend
1,Telephone and related communicatio...,cdotm_usna,jcdotm_usna,cdotqm_usna,dotm,dotm_quant,dotm_spend
2,Lotteries,csrolm_usna,jcsrolm_usna,csrolqm_usna,srolm,srolm_quant,srolm_spend
3,Foundations and grantmaking an...,csssfm_usna,jcsssfm_usna,csssfqm_usna,sssfm,sssfm_quant,sssfm_spend
4,Package tours,csrkm_usna,jcsrkm_usna,csrkqm_usna,srkm,srkm_quant,srkm_spend
...,...,...,...,...,...,...,...
125,Rental of tenant-occupied nonf...,cshtm_usna,jcshtm_usna,cshtqm_usna,shtm,shtm_quant,shtm_spend
126,New autos,cdmnm_usna,jcdmnm_usna,cdmnqm_usna,dmnm,dmnm_quant,dmnm_spend
127,Purchased meals and beverages ...,csfpm_usna,jcsfpm_usna,csfpqm_usna,sfpm,sfpm_quant,sfpm_spend
128,Imputed rental of owner-occupi...,csrdm_usna,jcsrdm_usna,csrdqm_usna,srdm,srdm_quant,srdm_spend


In [ ]:
raw_data = pd.read_stata(r"B:\OneDrive - DAZ\Desktop\Replicating Shapiro Final\raw_pce_data.dta")
raw_data = raw_data[(raw_data['time_month'] > '1959-01-01') & (raw_data['time_month'] < '2024-09-01')]
raw_data

In [ ]:
rename_dict = {}

for i, row in haver_codes.iterrows():
    rename_dict[row['Haver Code Expenditure']] = f"{row['PCE Category']} Expenditure"
    rename_dict[row['Haver Code Price']] = f"{row['PCE Category']} Price"
    rename_dict[row['Haver Code Quantity']] = f"{row['PCE Category']} Quantity"

raw_data = raw_data.set_index('time_month').T
raw_data = raw_data.rename(index=rename_dict)




quantity = raw_data[raw_data.index.str.endswith('Quantity')]
price = raw_data[raw_data.index.str.endswith('Price')]
expenditure = raw_data[raw_data.index.str.endswith('Expenditure')]


expenditure

In [107]:
with pd.ExcelWriter('Shapiro_Data.xlsx') as writer:
    quantity.to_excel(writer, sheet_name='Quantity Shapiro')
    price.to_excel(writer, sheet_name='Price Shapiro')
    expenditure.to_excel(writer, sheet_name='Expenditure Shapiro')


In [ ]:
# rename_map = {
#     'cm_usna':        'pce_spend',
#     'jcm_usna':       'pce',
#     'jcxfem_usna':    'core',
#     'cxfem_usna':     'core_spend',
#     'jcgxfem_usna':   'core_goods',
#     'jcsxem_usna':    'core_serv',
#     'cgxfem_usna':    'core_goods_spend',
#     'csxem_usna':     'core_serv_spend',
#     'jnpcfm_usna':    'pcfm',
#     'npcfm_usna':     'pcfm_spend',
#     'npcfqm_usna':    'pcfm_quant',
#     'cut_ip':     'util',
#     'ip_ip':     'IP'
# }

# PCE_data = PCE_data.rename(columns=rename_map)


# rename_map_2 = {
#     'jnpcfm_usna':    'pcfm',
#     'npcfm_usna':     'pcfm_spend',
#     'npcfqm_usna':    'pcfm_quant'
# }

# PCE_data = PCE_data.rename(columns=rename_map_2)
